In [1]:
!pip install segmentation_models_pytorch warmup_scheduler albumentations -q


In [2]:
import os
import sys
import random
from glob import glob
import warnings
import timm

import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2

sys.path.append('/kaggle/working/notebook/experiment/2d')
from script.helper import *
from script.dataset import *
from script.metrics import *
from script.trainer import *
from script.model import *
from script.loss import *
from script.scheduler import *


## config

In [3]:
class CFG:
    # ============== comp exp name =============
    comp_name = 'vesuvius'
    comp_dir_path = '/kaggle/input/'
    comp_folder_name = 'vesuvius-challenge-ink-detection'

    dataset_path = "/kaggle/working/dataset_train/"
    train_dataset_path = "/kaggle/working/dataset_train/nonflatten/29-34/"

    exp_name = os.getcwd().split('/')[-1]

    # ============== model cfg =============
    model_name = 'Unet'
    backbone = 'se_resnext50_32x4d'
    in_chans = 6

    # ============== pred target =============
    target_size = 1

    # ============== training cfg =============
    size = 16
    tile_size = 16
    stride = tile_size //1

    train_batch_size = 64  # 32
    valid_batch_size = train_batch_size * 2
    use_amp = True

    scheduler = 'GradualWarmupSchedulerV2'
    # scheduler = 'CosineAnnealingLR'

    epochs = 20  # 15

    warmup_factor = 10
    lr = 1e-4 / warmup_factor

    # ============== fold =============
    metric_direction = 'maximize'  # maximize, 'minimize'

    # ============== fixed =============
    pretrained = True
    inf_weight = 'best'  # 'best'

    min_lr = 1e-6
    weight_decay = 1e-6
    max_grad_norm = 1000

    print_freq = 50
    num_workers = 4

    seed = 42

    # ============== augmentation =============
    train_aug_list = [
        A.Resize(size, size),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        # A.RandomBrightnessContrast(p=0.75),
        # A.ShiftScaleRotate(p=0.75),
        # A.OneOf([
        #         A.GaussNoise(var_limit=[10, 50]),
        #         A.GaussianBlur(),
        #         A.MotionBlur(),
        #         ], p=0.4),
        # A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.5),
        # A.CoarseDropout(max_holes=1, max_width=int(size * 0.3), max_height=int(size * 0.3),
        #                 mask_fill_value=0, p=0.5),
        A.Normalize(
            mean=[0] * in_chans,
            std=[1] * in_chans,
            max_pixel_value=65535,
        ),
        ToTensorV2(transpose_mask=True),
    ]

    valid_aug_list = [
        A.Resize(size, size),
        A.Normalize(
            mean=[0] * in_chans,
            std=[1] * in_chans,
            max_pixel_value=65535,
        ),
        ToTensorV2(transpose_mask=True),
    ]


warnings.filterwarnings("ignore")
torch.backends.cudnn.benchmark = True  # type: ignore

set_seed(CFG.seed)
os.makedirs(f'./{CFG.exp_name}/', exist_ok=True)


## Dataset

In [4]:
def preprocess(image, fragment_i, split_i):
    # image=np.clip(image, a_min=0.15,a_max=0.7)
    return image


In [5]:
def get_train_valid_dataset(valid_fragment_i, valid_split_i, cfg, preprocess, label_prefix="inklabels"):
    train_images = []
    train_labels = []
    train_masks = []

    valid_images = []
    valid_labels = []
    valid_xyxys = []

    for stack_path in glob(f"{cfg.train_dataset_path}/*"):
        fragment_i = int(stack_path.split(".")[-2].split("/")[-1].split("_")[0])
        split_i = int(stack_path.split(".")[-2].split("/")[-1].split("_")[1])
        image, label, mask = read_image_label_mask(stack_path, cfg, preprocess, fragment_i, split_i, label_prefix)
        x1_list = list(range(0, image.shape[1]-cfg.tile_size+1, cfg.stride))
        y1_list = list(range(0, image.shape[0]-cfg.tile_size+1, cfg.stride))

        for y1 in y1_list:
            for x1 in x1_list:
                y2 = y1 + cfg.tile_size
                x2 = x1 + cfg.tile_size

                if (fragment_i == valid_fragment_i) & (split_i == valid_split_i):
                    valid_images.append(image[y1:y2, x1:x2])
                    label_ = np.where(np.any(np.array(label[y1:y2, x1:x2, None]) == 1), 1, 0)
                    valid_labels.append(label_)

                    valid_xyxys.append([x1, y1, x2, y2])
                else:
                    train_images.append(image[y1:y2, x1:x2])
                    label_ = np.where(np.any(np.array(label[y1:y2, x1:x2, None]) == 1), 1, 0)
                    train_labels.append(label_)
                    train_masks.append(mask[y1:y2, x1:x2, None])
    valid_xyxys = np.stack(valid_xyxys)
    return train_images, train_labels, train_masks, valid_images, valid_labels, valid_xyxys

In [6]:
# confirmation

valid_fragment_i = 1
valid_split_i = 0

train_images, train_labels, train_masks, valid_images, valid_labels, valid_xyxys = get_train_valid_dataset(valid_fragment_i, valid_split_i, CFG, preprocess)

print(f"""
{len(train_images) = }
{train_images[0].shape = }
{train_labels[0].shape = }
{train_masks[0].shape = }

{len(valid_images) = }
{valid_images[0].shape = }
{valid_labels[0].shape = }
""")

# id = random.randint(0, len(train_labels))
# visualize_train_images(id, train_images, train_labels, train_masks)



len(train_images) = 708764
train_images[0].shape = (16, 16, 6)
train_labels[0].shape = ()
train_masks[0].shape = (16, 16, 1)

len(valid_images) = 202752
valid_images[0].shape = (16, 16, 6)
valid_labels[0].shape = ()



In [7]:
class CustomDataset(Dataset):
    def __init__(self, images, cfg, labels=None, transform=None):
        self.images = images
        self.cfg = cfg
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]  # type: ignore

        if self.transform:
            data = self.transform(image=image)
            image = data['image']
        label=torch.tensor(label).unsqueeze(0).float()
        return image, label

In [8]:
train_dataset = CustomDataset(
    train_images, CFG, labels=train_labels, transform=get_transforms(data='train', cfg=CFG))

valid_dataset = CustomDataset(
    valid_images, CFG, labels=valid_labels, transform=get_transforms(data='valid', cfg=CFG))

train_loader = DataLoader(train_dataset,
                            batch_size=CFG.train_batch_size,
                            shuffle=True,
                            num_workers=CFG.num_workers, pin_memory=True, drop_last=True,
                            )
valid_loader = DataLoader(valid_dataset,
                            batch_size=CFG.valid_batch_size,
                            shuffle=False,
                            num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

## main

In [9]:
from timm.utils import AverageMeter
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

model = timm.create_model("efficientnet_b0", pretrained=True, num_classes=1, in_chans=CFG.in_chans,).cuda()

# 最適化手法
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-4)

# 損失関数
criterion = torch.nn.BCEWithLogitsLoss()

# ログ記録用の変数
history = {"train": [], "test": []}
scaler = torch.cuda.amp.GradScaler()

In [10]:
# 学習回数
for epoch in range(5):
    preds_list=[]
    target_list=[]
    print("\nEpoch:", epoch)

    # 学習
    model.train()
    train_loss = AverageMeter()
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        image = batch[0].cuda() #(batch_size, channel, size, size)
        label = batch[1].cuda() #(batch_size)
        
        with torch.cuda.amp.autocast():  # type: ignore
            preds = model(image)
            loss = criterion(preds, label)
            scaler.scale(loss).backward()  # type: ignore
            scaler.step(optimizer)
            scaler.update()
        train_loss.update(val = loss.item(), n = len(image))

    # 検証
    model.eval()
    test_loss = AverageMeter()
    with torch.no_grad():
        for batch in tqdm(valid_loader):
            image = batch[0].cuda() #(batch_size, channel, size, size)
            label = batch[1].cuda() #(batch_size)
            preds = model(image) #(batch_size, num_class)
            loss = criterion(preds, label)
            # print(label)
            test_loss.update(val = loss.item(), n = len(image))
            preds=preds.squeeze(-1).cpu().sigmoid().numpy()
            
            preds_prob=np.where(preds>0.5, 1, 0).tolist()
            preds_list+=preds_prob
            
            flat_list = [item for sublist in label.cpu().numpy().tolist() for item in sublist]
            target_list+=flat_list
            # print(accuracy_score(preds_list, target_list))
    matrix = confusion_matrix(target_list, preds_list)
    print(matrix)

    # 誤差出力
    # print(train_loss.avg)
    # print(test_loss.avg)
    history["train"].append(train_loss.avg)
    history["test"].append(test_loss.avg)


Epoch: 0


100%|██████████| 1584/1584 [00:10<00:00, 156.95it/s]


[[178335    194]
 [ 24219      4]]

Epoch: 1


  3%|▎         | 315/11074 [00:06<03:58, 45.17it/s]


KeyboardInterrupt: 

In [ ]:
print(matrix)

[]


array([], shape=(0, 0), dtype=int64)